## Selection of Brainwide map sessions with good QC

In [1]:
"""
Check sessions available with dlc data
@author: Ines
"""
import pandas as pd
import pickle 
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
from one.api import ONE
from uuid import UUID
import json

# Get my functions
functions_path =  '/home/ines/repositories/representation_learning_variability/Functions/'
# functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/'

os.chdir(functions_path)
from one_functions_generic import query_subjects_interest, subjects_interest_data, prepro

one = ONE()

In [2]:

def get_sess_metadata(input_sessions):
    
    """ Download session data """
    # Search sessions of interest
    mouse_project='ibl_neuropixel_brainwide_01'# GETS DATA PER SESSION
    sessions = one.search(project=mouse_project, details=True)
    session_eids = sessions[0]
    session_details = sessions[1]

    # Initialize dataframe to collect all data
    trials_df = pd.DataFrame(columns=['subject_nickname', 'session_date', 'task_protocol', 'session_uuid'], index=range(len(input_sessions)))
    # Loop through sessions
    for i, ses in enumerate(list(input_sessions)):
        index = np.where(np.array(session_eids) == ses)[0][0]
        # Save session details
        trials_df['subject_nickname'][i] = session_details[index]['subject']
        trials_df['session_date'][i] = session_details[index]['date']
        trials_df['task_protocol'][i] = session_details[index]['task_protocol']
        trials_df['session_uuid'][i] = ses
        
    return trials_df


In [5]:
sess_metadata = get_sess_metadata(pass_qc)
sess_metadata

,subject_nickname,session_date,task_protocol,session_uuid
0,NYU-37,2021-01-25,_iblrig_tasks_ephysChoiceWorld6.4.2,7af49c00-63dd-4fed-b2e0-1b3bd945b20b
1,NYU-37,2021-01-26,_iblrig_tasks_ephysChoiceWorld6.4.2,f88d4dd4-ccd7-400e-9035-fa00be3bcfa8
2,NYU-37,2021-01-27,_iblrig_tasks_ephysChoiceWorld6.4.2,83d85891-bd75-4557-91b4-1cbb5f8bfc9d
3,NYU-37,2021-01-30,_iblrig_tasks_ephysChoiceWorld6.4.2,aec5d3cc-4bb2-4349-80a9-0395b76f04e2
4,NYU-39,2021-05-13,_iblrig_tasks_ephysChoiceWorld6.4.2,ee212778-3903-4f5b-ac4b-a72f22debf03
...,...,...,...,...
171,ibl_witten_29,2021-06-10,_iblrig_tasks_ephysChoiceWorld6.4.2,e9fc0a2d-c69d-44d1-9fa3-314782387cae
172,ibl_witten_29,2021-06-16,_iblrig_tasks_ephysChoiceWorld6.4.2,821f1883-27f3-411d-afd3-fb8241bbc39a
173,ibl_witten_29,2021-06-17,_iblrig_tasks_ephysChoiceWorld6.4.2,69a0e953-a643-4f0e-bb26-dc65af3ea7d7
174,ibl_witten_29,2021-06-18,_iblrig_tasks_ephysChoiceWorld6.4.2,86b6ba67-c1db-4333-add0-f8105ea6e363


### Get QC of all sessions

In [3]:
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Brainwidemap/'
# data_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Brainwidemap/'

os.chdir(data_path)
qc = pd.read_csv("Brainwidemap_QC - behavior_data.csv")


### Select sessions that pass QC for left DLC and left video

Video and DLC are set to pass if all tests in the respective categories pass as well

In [36]:
pass_qc = qc.loc[(qc['leftVideo']=='PASS') & (qc['leftDLC']=='PASS'), 'Session ID']
print(str(len(pass_qc)) + ' sessions pass QC')

25 sessions pass QC


Some sessions pass leftDLC but not leftVideo. Why would this be?

In [4]:
pass_qc = qc.loc[(qc['leftDLC']=='PASS'), 'Session ID']
print(str(len(pass_qc)) + ' sessions pass QC')

176 sessions pass QC


### Video that is both good on the left and right sides

In [7]:
pass_qc = qc.loc[(qc['leftVideo']=='PASS') & (qc['leftDLC']=='PASS') &
                 (qc['rightVideo']=='PASS') & (qc['rightDLC']=='PASS'), 'Session ID']
print(str(len(pass_qc)) + ' sessions pass QC')

6 sessions pass QC


Look into the extended QC of a passed session

In [24]:
example_session = list(pass_qc)[4]
one = ONE()
extended_qc = one.get_details(example_session, True)['extended_qc']
extended_qc


{'task': 'WARNING',
 'dlcBody': 'PASS',
 'dlcLeft': 'PASS',
 'behavior': 1,
 'dlcRight': 'FAIL',
 'videoBody': 'WARNING',
 'videoLeft': 'PASS',
 'videoRight': 'PASS',
 '_task_iti_delays': 0.1706896551724138,
 '_videoBody_focus': True,
 '_videoLeft_focus': True,
 '_videoRight_focus': True,
 '_task_goCue_delays': 0.9965576592082617,
 '_task_trial_length': 0.9965576592082617,
 '_task_stimOn_delays': 0.9931153184165232,
 '_videoBody_position': True,
 '_videoLeft_position': True,
 '_dlcBody_paw_far_nan': None,
 '_dlcLeft_paw_far_nan': True,
 '_task_n_trial_events': 0.9913941480206541,
 '_task_reward_volumes': 1.0,
 '_task_stimOff_delays': 0.9931153184165232,
 '_videoBody_framerate': [True, 30.063],
 '_videoBody_pin_state': [True, 0, 0],
 '_videoLeft_framerate': [True, 60.006],
 '_videoLeft_pin_state': [True, 0, 0],
 '_videoRight_position': True,
 '_dlcBody_mean_in_bbox': True,
 '_dlcLeft_mean_in_bbox': True,
 '_dlcRight_paw_far_nan': False,
 '_task_audio_pre_trial': 1.0,
 '_task_errorCue_de

In [6]:
save_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/'
os.chdir(save_path)
pickle.dump(sess_metadata, open("good_dlc_bwm_sessions_metadata", "wb"))